In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import header
paths = header.setup_environment()
paths['daily']=os.path.join(paths['WELLDATA_TOP'], '05_DAILY_RC') # raw and corrected, and done so using appropriate well barometer
if not os.path.isdir(paths['daily']):
    os.makedirs(paths['daily'])
import platform
if platform.system()=='Darwin':
    INPUTDIR = '/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3_Project_Documents/NASAprojects/201602_Rocket_Seismology/DATA/2022_DATA/WellData/MERGED'
else:
    INPUTDIR = os.path.join(paths['new_data'], '03_merge_inventories')
import libWellData as LLE
transducersDF = LLE.get_transducers_dataframe(paths)
display(transducersDF)

# Load in the summary of all files - raw 4 hourly data 
dfall2 = pd.read_csv(os.path.join(INPUTDIR, 'all2.csv'))
#display(dfall2['subdir'])

LLE.qc_dataframe(dfall2)

# Split in baro, 20Hz, 100Hz
all_dataframes = LLE.split_by_subdir(dfall2, verbose=True)
#print(all_dataframes)


Linux


,serial,Fs,sensor,shielding,range_kPa_low,range_kPa_high,media,type,model,set_depth_ft,id,well,tmean,dig0,gf,tt,tt0,tf,bp,bp0
0,AirPressureShallow,100,barometer,none,100,100,air,pressure,Keller 0507.01401.051311.07,3.81,6S.02374.88.HDH,shallow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1226420,100,vibrating_wire,none,70,170,air,level,Geokon 4500AL,4.46,6S.12264.20.HDD,shallow,30.68,9751.0,-0.006458,21.6,21.3,-0.008795,14.504,14.298
2,1226423,20,vibrating_wire,foam,70,170,water,level,Geokon 4500AL,-5.83,6S.12264.23.BDD,shallow,28.40,9605.0,-0.006347,21.6,22.2,-0.004197,14.504,14.298
3,1226419,100,vibrating_wire,foam,70,170,water,level,Geokon 4500AL,-6.71,6S.12264.19.HDD,shallow,27.57,10040.0,-0.006441,21.6,21.1,-0.010870,14.504,14.298
4,1226421,100,vibrating_wire,none,70,170,water,level,Geokon 4500AL,-7.71,6S.12264.21.HDD,shallow,27.60,9787.0,-0.006724,21.6,21.3,-0.001145,14.504,14.298
5,AirPressureIntermediate,100,barometer,none,100,100,air,pressure,Keller 0507.01401.051311.07,3.86,6I.0XXXX.XX.HDH,intermediate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1226429,100,vibrating_wire,none,70,170,air,level,Geokon 4500AL,4.11,6I.12264.29.HDD,intermediate,30.60,9800.0,-0.006428,22.6,21.6,-0.002384,14.367,14.298
7,2151692,20,vibrating_wire,foam,70,170,water,level,Geokon 4500AL,-9.14,6I.21516.92.BDD,intermediate,25.88,9459.0,-0.008038,22.8,21.8,0.007666,14.296,14.388
8,2151691,100,vibrating_wire,foam,70,170,water,level,Geokon 4500AL,-18.31,6I.21516.91.HDD,intermediate,25.70,9414.0,-0.008142,22.8,21.5,0.008742,14.296,14.388
9,2149882,100,vibrating_wire,none,70,170,water,level,Geokon 4500AL,-19.14,6I.21498.82.HDD,intermediate,25.96,9734.0,-0.008075,20.7,21.3,0.000675,14.602,14.389


Baro
**********
Displaying  Baro


,1226421,1226419,1226420,2149882,2151691,1226429,uploaddate,subdir,basename,1226419_temp,...,1226420_std,2151691_std,2149882_std,1226429_std,DynStdDev(6),1226423,2151692,AirPressureShallow,AirPressureDeep,datetime
3,NaN,NaN,NaN,NaN,NaN,NaN,20220826,Baro,100hz_Baro_Sensors_20220721135421_000.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.773304,14.766960,2022-07-21 13:54:21.790
4,NaN,NaN,NaN,NaN,NaN,NaN,20220826,Baro,100hz_Baro_Sensors_20220721160000_001.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.754274,14.752241,2022-07-21 16:00:00.010
10,NaN,NaN,NaN,NaN,NaN,NaN,20220826,Baro,100hz_Baro_Sensors_20220721200000_002.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.754250,14.748769,2022-07-21 20:00:00.010
15,NaN,NaN,NaN,NaN,NaN,NaN,20220826,Baro,100hz_Baro_Sensors_20220722000000_003.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.759185,14.758797,2022-07-22 00:00:00.010
19,NaN,NaN,NaN,NaN,NaN,NaN,20220826,Baro,100hz_Baro_Sensors_20220722040000_004.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.765824,14.759901,2022-07-22 04:00:00.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2258,NaN,NaN,NaN,NaN,NaN,NaN,20221202,Baro,100hz_Baro_Sensors_20221201120000_119.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.877370,14.876700,2022-12-01 12:00:00.010
2262,NaN,NaN,NaN,NaN,NaN,NaN,20221202,Baro,100hz_Baro_Sensors_20221201160000_120.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.859900,14.857960,2022-12-01 16:00:00.010
2265,NaN,NaN,NaN,NaN,NaN,NaN,20221202,Baro,100hz_Baro_Sensors_20221201200000_121.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.880270,14.873140,2022-12-01 20:00:00.010
2268,NaN,NaN,NaN,NaN,NaN,NaN,20221202,Baro,100hz_Baro_Sensors_20221202000000_122.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.886800,14.880480,2022-12-02 00:00:00.010


20hz
**********
Displaying  20hz


,1226421,1226419,1226420,2149882,2151691,1226429,uploaddate,subdir,basename,1226419_temp,...,1226420_std,2151691_std,2149882_std,1226429_std,DynStdDev(6),1226423,2151692,AirPressureShallow,AirPressureDeep,datetime
1,NaN,NaN,NaN,NaN,NaN,NaN,20220826,20hz,20hz_Sensors_20Hz_20220721134750_000.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,22819.189453,15339.369141,NaN,NaN,2022-07-21 13:47:50.000
2,NaN,NaN,NaN,NaN,NaN,NaN,20220826,20hz,20hz_Sensors_1Sec_20220721134750_000.csv,27.963470,...,0.027189,NaN,NaN,NaN,NaN,9078.993164,8428.339844,NaN,NaN,2022-07-21 13:47:50.000
6,NaN,NaN,NaN,NaN,NaN,NaN,20220826,20hz,20hz_Sensors_20Hz_20220721160000_001.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9082.031250,8430.769531,NaN,NaN,2022-07-21 16:00:00.050
8,NaN,NaN,NaN,NaN,NaN,NaN,20220826,20hz,20hz_Sensors_1Sec_20220721160001_001.csv,27.960571,...,0.028026,NaN,NaN,NaN,NaN,9082.081055,8430.733398,NaN,NaN,2022-07-21 16:00:01.000
11,NaN,NaN,NaN,NaN,NaN,NaN,20220826,20hz,20hz_Sensors_20Hz_20220721200000_002.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9081.624023,8430.257812,NaN,NaN,2022-07-21 20:00:00.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,NaN,NaN,NaN,NaN,NaN,NaN,20221202,20hz,20hz_Sensors_20Hz_20221201120000_119.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9055.340820,8414.737305,NaN,NaN,2022-12-01 12:00:00.050
2263,NaN,NaN,NaN,NaN,NaN,NaN,20221202,20hz,20hz_Sensors_20Hz_20221201160000_120.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9060.303711,8418.320312,NaN,NaN,2022-12-01 16:00:00.050
2266,NaN,NaN,NaN,NaN,NaN,NaN,20221202,20hz,20hz_Sensors_20Hz_20221201200000_121.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9056.477539,8415.290039,NaN,NaN,2022-12-01 20:00:00.050
2269,NaN,NaN,NaN,NaN,NaN,NaN,20221202,20hz,20hz_Sensors_20Hz_20221202000000_122.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,9055.333984,8414.057617,NaN,NaN,2022-12-02 00:00:00.050


100hz
**********
Displaying  100hz


,1226421,1226419,1226420,2149882,2151691,1226429,uploaddate,subdir,basename,1226419_temp,...,1226420_std,2151691_std,2149882_std,1226429_std,DynStdDev(6),1226423,2151692,AirPressureShallow,AirPressureDeep,datetime
0,NaN,NaN,NaN,NaN,NaN,NaN,20220826,100hz,100hz_Sensors_1Sec_20220721133627_000.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-21 13:36:27.000
5,9167.110352,9489.996094,9703.252930,8627.779297,8358.027344,9770.319336,20220826,100hz,100hz_Sensors_100Hz_20220721160000_001.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-21 16:00:00.010
7,9167.369141,9489.804688,9702.906250,8627.558594,8358.027344,9770.466797,20220826,100hz,100hz_Sensors_1Sec_20220721160001_001.csv,27.561035,...,0.207875,0.274275,0.250625,0.196379,0.277220,NaN,NaN,NaN,NaN,2022-07-21 16:00:01.000
9,9167.326172,9489.264648,9703.137695,8626.943359,8357.288086,9770.840820,20220826,100hz,100hz_Sensors_100Hz_20220721200000_002.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-21 20:00:00.010
12,9166.939453,9489.232422,9703.239258,8627.102539,8357.619141,9770.675781,20220826,100hz,100hz_Sensors_1Sec_20220721200001_002.csv,27.521179,...,0.188281,0.233037,0.248869,0.196529,0.283193,NaN,NaN,NaN,NaN,2022-07-21 20:00:01.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,9143.607000,9463.829000,9707.100000,8611.594000,8341.694000,9767.984000,20221202,100hz,100hz_Sensors_100Hz_20221201120000_119.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01 12:00:00.010
2261,9148.351000,9468.346000,9707.489000,8614.964000,8345.216000,9769.956000,20221202,100hz,100hz_Sensors_100Hz_20221201160000_120.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01 16:00:00.010
2264,9144.639000,9464.844000,9706.258000,8612.217000,8341.727000,9767.146000,20221202,100hz,100hz_Sensors_100Hz_20221201200000_121.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-01 20:00:00.010
2267,9143.237000,9463.555000,9705.298000,8611.196000,8340.690000,9767.320000,20221202,100hz,100hz_Sensors_100Hz_20221202000000_122.csv,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 00:00:00.010


In [2]:
startt = dfall2['datetime'].min()
startt = pd.to_datetime('2022-08-04')
endt = dfall2['datetime'].max()+pd.Timedelta(hours=24)
thisdate = pd.to_datetime(f'{startt.year}-{startt.month:02d}-{startt.day:02d}')
lod = []

#csvfile = os.path.join(INPUTDIR, 'dailyshifts_nobaro.csv')
'''
if os.path.isfile(csvfile):
    dfshifts = pd.read_csv(csvfile, index_col=None)
    lod = dfshifts.to_dict('records')
    endt = pd.to_datetime(dfshifts.iloc[-1]['datetime'])+pd.Timedelta(hours=24)
    thisdate = pd.to_datetime(f'{startt.year}-{startt.month:02d}-{startt.day:02d}')
'''
print(startt, endt)
while thisdate < endt:
    print(thisdate)

    #correctedAllSensorsPSI, dcshifts, sensor_depths = LLE.process_day(dfall2, thisdate, INPUTDIR, transducersDF, apcol=None)
    correctedAllSensorsPSI = LLE.process_day(dfall2, thisdate, INPUTDIR, transducersDF, apcol='well')
    #print('dcshifts=',dcshifts)
    #print('sensor_depths=',sensor_depths)
    if isinstance(correctedAllSensorsPSI, pd.DataFrame):
        correctedAllSensorsPSI.to_pickle(os.path.join(paths['daily'], f'{thisdate.strftime("%Y%m%d")}.pkl'))
    """
    if dcshifts or not sensor_depths.empty:
        dcshifts['datetime']=thisdate.strftime("%Y-%m-%d")
        for key, value in sensor_depths.items():
            dcshifts[key] = value
        lod.append(dcshifts)

        # could just do this once at end of loop - but good to see it build each day
        dfshifts = pd.DataFrame(lod)
        dfshifts.to_csv(csvfile, index=None)
        display(dfshifts)
    """
    thisdate = thisdate + pd.Timedelta(hours=24)



2022-08-04 00:00:00 2022-12-03 08:24:14
2022-08-04 00:00:00
existing day pickle files found - loading these is faster
reading raw_2022-08-04_20hz.pkl into dfdata[20hz]
reading raw_2022-08-04_baro.pkl into dfdata[baro]
no 100hz
Columns in dfmerged: 
 Index(['AirPressureShallow', 'AirPressureIntermediate', 'datetime', '1226423',
       '2151692', '1226423_temp', '2151692_temp', '2151692_std'],
      dtype='object')
Applying height correction for AirPressureShallow
Applying height correction for AirPressureIntermediate
- height corrected


,AirPressureShallow,AirPressureIntermediate,datetime,1226423,2151692,1226423_temp,2151692_temp,2151692_std
0,14.779969,14.779473,2022-08-04 00:00:00.010,NaN,NaN,NaN,NaN,NaN
1,14.783734,14.774851,2022-08-04 00:00:00.020,NaN,NaN,NaN,NaN,NaN
2,14.781873,14.775402,2022-08-04 00:00:00.030,NaN,NaN,NaN,NaN,NaN
3,14.781173,14.777129,2022-08-04 00:00:00.040,NaN,NaN,NaN,NaN,NaN
4,14.782248,14.777708,2022-08-04 00:00:00.050,9136.987305,8480.850586,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
8640093,14.791890,14.789597,2022-08-04 23:59:59.960,9142.675586,8485.820508,NaN,NaN,NaN
8640094,14.794045,14.789606,2022-08-04 23:59:59.970,9142.686133,8485.816797,NaN,NaN,NaN
8640095,14.796025,14.789389,2022-08-04 23:59:59.980,9142.696680,8485.813086,NaN,NaN,NaN
8640096,14.793107,14.788370,2022-08-04 23:59:59.990,9142.707227,8485.809375,NaN,NaN,NaN


Applying calibration correction for 1226423
mean before: 9140.139119025356, after: 2.950472011546075
min, max before: 9135.40625, 9144.5361328125
min, max after: 2.922564165039063, 2.98051153125
Applying temperature correction for 1226423
Applying air pressure correction for 1226423
Applying calibration correction for 2151692
mean before: 8483.415113188237, after: 7.841751320192963
min, max before: 8479.794921875, 8487.2841796875
min, max after: 7.810651763671875, 7.87085041796875
Applying temperature correction for 2151692
Applying air pressure correction for 2151692
- all corrected


,AirPressureShallow,AirPressureIntermediate,datetime,1226423,2151692,1226423_temp,2151692_temp,2151692_std,1226423_corrected,2151692_corrected
0,14.779969,14.779473,2022-08-04 00:00:00.010,NaN,NaN,28.4,25.88,NaN,NaN,NaN
1,14.783734,14.774851,2022-08-04 00:00:00.020,NaN,NaN,28.4,25.88,NaN,NaN,NaN
2,14.781873,14.775402,2022-08-04 00:00:00.030,NaN,NaN,28.4,25.88,NaN,NaN,NaN
3,14.781173,14.777129,2022-08-04 00:00:00.040,NaN,NaN,28.4,25.88,NaN,NaN,NaN
4,14.782248,14.777708,2022-08-04 00:00:00.050,2.970477,7.862365,28.4,25.88,NaN,2.460207,7.499394
...,...,...,...,...,...,...,...,...,...,...
8640093,14.791890,14.789597,2022-08-04 23:59:59.960,2.934373,7.822417,28.4,25.88,NaN,2.414462,7.449804
8640094,14.794045,14.789606,2022-08-04 23:59:59.970,2.934306,7.822447,28.4,25.88,NaN,2.412239,7.447679
8640095,14.796025,14.789389,2022-08-04 23:59:59.980,2.934239,7.822476,28.4,25.88,NaN,2.410193,7.445729
8640096,14.793107,14.788370,2022-08-04 23:59:59.990,2.934172,7.822506,28.4,25.88,NaN,2.413044,7.448677


2022-08-05 00:00:00
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/Baro/100hz_Baro_Sensors_20220805000000_087.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/20hz/20hz_Sensors_20Hz_20220805000000_087.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/20hz/20hz_Sensors_1Sec_20220805000001_087.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/Baro/100hz_Baro_Sensors_20220805040000_088.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/20hz/20hz_Sensors_20Hz_20220805040000_088.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/20hz/20hz_Sensors_1Sec_20220805040001_088.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/Baro/100hz_Baro_Sensors_20220805080000_089.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20220826/20hz/20hz_Sensors_20Hz_2022080508

...

2022-12-02 00:00:00
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/100hz/100hz_Sensors_100Hz_20221202000000_122.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/Baro/100hz_Baro_Sensors_20221202000000_122.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/20hz/20hz_Sensors_20Hz_20221202000000_122.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/Baro/100hz_Baro_Sensors_20221202082411_123.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/100hz/100hz_Sensors_100Hz_20221202082414_123.csv
- loading
/data/KSC/EROSION/fromdropboxinventory/03_merge_inventories/20221202/20hz/20hz_Sensors_20Hz_20221202082414_123.csv
- loading
saving to raw_2022-12-02_100hz.pkl
saving to raw_2022-12-02_baro.pkl
saving to raw_2022-12-02_20hz.pkl
Columns in dfmerged: 
 Index(['1226421', '1226419', '1226420', '2149882', '2151691', '1226429',
       'datetime', 'Air

,1226421,1226419,1226420,2149882,2151691,1226429,datetime,AirPressureShallow,AirPressureDeep,1226423,2151692,1226420_corrected,1226429_corrected
0,9143.237,9463.555,0.297480,8611.196,8340.690,0.212220,2022-12-02 00:00:00.010,14.888796,14.88048,NaN,NaN,14.510646,14.486611
1,9143.182,9463.455,0.299043,8610.800,8340.731,0.213435,2022-12-02 00:00:00.020,14.885406,14.88247,NaN,NaN,14.512209,14.487826
2,9143.638,9463.338,0.302633,8610.891,8340.407,0.219959,2022-12-02 00:00:00.030,14.884706,14.88260,NaN,NaN,14.515800,14.494350
3,9143.298,9463.766,0.296382,8610.696,8341.092,0.214740,2022-12-02 00:00:00.040,14.884866,14.87970,NaN,NaN,14.509549,14.489131
4,9143.381,9463.280,0.297648,8610.815,8340.629,0.215621,2022-12-02 00:00:00.050,14.884986,14.87972,9055.333984,8414.057617,14.510814,14.490011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028777,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.780,14.888416,14.88396,NaN,NaN,NaN,NaN
3028778,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.790,14.888626,14.88425,NaN,NaN,NaN,NaN
3028779,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.800,14.888886,14.88344,NaN,NaN,NaN,NaN
3028780,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.810,14.890806,14.88383,NaN,NaN,NaN,NaN


Applying calibration correction for 1226421
mean before: 9142.928960848456, after: 4.330733667254984
min, max before: 9121.186, 9321.41
min, max after: 3.130627160000001, 4.476933336000002
Applying temperature correction for 1226421
Applying air pressure correction for 1226421
Applying calibration correction for 1226419
mean before: 9463.055559603676, after: 3.716099140592732
min, max before: 2331.669, 9464.514
min, max after: 3.7067053260000047, 49.649359970999996
Applying temperature correction for 1226419
Applying air pressure correction for 1226419
Applying calibration correction for 2149882
mean before: 8610.412878735984, after: 9.072966004206966
min, max before: 514.9386, 8636.883
min, max after: 8.859219775000001, 74.443920805
Applying temperature correction for 2149882
Applying air pressure correction for 2149882
Applying calibration correction for 2151691
mean before: 8340.340012782739, after: 8.741739615922942
min, max before: 8332.885, 8341.567
min, max after: 8.731749486000

,1226421,1226419,1226420,2149882,2151691,1226429,datetime,AirPressureShallow,AirPressureDeep,1226423,2151692,1226420_corrected,1226429_corrected,1226421_corrected,1226419_corrected,2149882_corrected,2151691_corrected,1226423_corrected,2151692_corrected
0,4.328662,3.712882,0.297480,9.066642,8.738890,0.212220,2022-12-02 00:00:00.010,14.888796,14.88048,NaN,NaN,14.510646,14.486611,3.730653,3.051757,8.569993,8.274811,NaN,NaN
1,4.329032,3.713526,0.299043,9.069840,8.738556,0.213435,2022-12-02 00:00:00.020,14.885406,14.88247,NaN,NaN,14.512209,14.487826,3.734413,3.055792,8.576581,8.277867,NaN,NaN
2,4.325966,3.714280,0.302633,9.069105,8.741194,0.219959,2022-12-02 00:00:00.030,14.884706,14.88260,NaN,NaN,14.515800,14.494350,3.732047,3.057245,8.576546,8.281205,NaN,NaN
3,4.328252,3.711523,0.296382,9.070680,8.735617,0.214740,2022-12-02 00:00:00.040,14.884866,14.87970,NaN,NaN,14.509549,14.489131,3.734173,3.054328,8.577960,8.275467,NaN,NaN
4,4.327694,3.714654,0.297648,9.069719,8.739387,0.215621,2022-12-02 00:00:00.050,14.884986,14.87972,9055.333984,8.399247,14.510814,14.490011,3.733495,3.057339,8.576879,8.279117,9054.720977,7.933538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3028777,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.780,14.888416,14.88396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3028778,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.790,14.888626,14.88425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3028779,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.800,14.888886,14.88344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3028780,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-02 08:24:47.810,14.890806,14.88383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2022-12-03 00:00:00
No 100hz, no baro
